# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City")


# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] <=28) & (city_data_df["Humidity"]<50)]

# Drop any rows with null values
city_data_df = city_data_df.dropna(how="any")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
30,30,midwest city,35.4495,-97.3967,9.12,33,75,6.17,US,1666108247
36,36,drochia,48.0350,27.8161,13.39,30,73,3.46,MD,1666108253
44,44,collierville,35.0420,-89.6645,7.11,47,0,5.66,US,1666108151


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
4,lanzhou,CN,36.0564,103.7922,48,
30,midwest city,US,35.4495,-97.3967,33,
36,drochia,MD,48.0350,27.8161,30,
44,collierville,US,35.0420,-89.6645,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
lanzhou - nearest hotel: 西园宾馆
midwest city - nearest hotel: No hotel found
drochia - nearest hotel: Hotel Confort
collierville - nearest hotel: Days Inn & Suites by Wyndham Collierville Germantown Area
nalut - nearest hotel: No hotel found
rafaela - nearest hotel: Parra Hotel & Suites
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
arman - nearest hotel: No hotel found
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
heihe - nearest hotel: Санкт-Петербург
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
palencia - nearest hotel: No hotel found
yidu - nearest hotel: No hotel found
cedar city - nearest hotel: Baymont Inn & Suites
baruun-urt - nearest hotel: No hotel found
brenham - nearest hotel: Ant Street Inn
almaznyy - nearest hotel: Зверево
sur - nearest hotel: Sur Hotel
avera - nearest hotel: No hotel found
challapata - nearest hotel: Residencial Marian
neepawa - nearest hotel: Vivian Mo

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
30,midwest city,US,35.4495,-97.3967,33,No hotel found
36,drochia,MD,48.0350,27.8161,30,Hotel Confort
44,collierville,US,35.0420,-89.6645,47,Days Inn & Suites by Wyndham Collierville Germ...
...,...,...,...,...,...,...
554,freeport,US,40.6576,-73.5832,46,The Freeport Inn and Marina
555,bulgan,MN,48.8125,103.5347,40,Bulgan Hotel
557,saryg-sep,RU,51.5000,95.6000,44,No hotel found
560,changli,CN,39.7042,119.1506,33,客房


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])


# Display the map
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)